In [1]:
import cv2
import mediapipe as mp
import numpy as np


In [16]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# For static images:
IMAGE_FILES = []
BG_COLOR = (192, 192, 192)  # gray
# with mp_pose.Pose(
#         static_image_mode=True,
#         model_complexity=2,
#         enable_segmentation=True,
#         min_detection_confidence=0.5) as pose:
#     for idx, file in enumerate(IMAGE_FILES):
#         image = cv2.imread(file)
#         image_height, image_width, _ = image.shape
#         # Convert the BGR image to RGB before processing.
#         results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#         if not results.pose_landmarks:
#             continue
#         print(
#             f'Nose coordinates: ('
#             f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width}, '
#             f'{results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_height})'
#         )

#         annotated_image = image.copy()
#         # Draw segmentation on the image.
#         # To improve segmentation around boundaries, consider applying a joint
#         # bilateral filter to "results.segmentation_mask" with "image".
#         condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
#         bg_image = np.zeros(image.shape, dtype=np.uint8)
#         bg_image[:] = BG_COLOR
#         annotated_image = np.where(condition, annotated_image, bg_image)
#         # Draw pose landmarks on the image.
#         mp_drawing.draw_landmarks(
#             annotated_image,
#             results.pose_landmarks,
#             mp_pose.POSE_CONNECTIONS,
#             landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
#         cv2.imwrite('/tmp/annotated_image' +
#                     str(idx) + '.png', annotated_image)
#         # Plot pose world landmarks.
#         mp_drawing.plot_landmarks(
#             results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)


# with mp_hands.Hands(
#         static_image_mode=True,
#         max_num_hands=2,
#         min_detection_confidence=0.5) as hands:
#     for idx, file in enumerate(IMAGE_FILES):
#         # Read an image, flip it around y-axis for correct handedness output (see
#         # above).
#         image = cv2.flip(cv2.imread(file), 1)
#         # Convert the BGR image to RGB before processing.
#         results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#         # Print handedness and draw hand landmarks on the image.
#         print('Handedness:', results.multi_handedness)
#         if not results.multi_hand_landmarks:
#             continue
#         image_height, image_width, _ = image.shape
#         annotated_image = image.copy()
#         for hand_landmarks in results.multi_hand_landmarks:
#             print('hand_landmarks:', hand_landmarks)
#             print(
#                 f'Index finger tip coordinates: (',
#                 f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
#                 f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
#             )
#             mp_drawing.draw_landmarks(
#                 annotated_image,
#                 hand_landmarks,
#                 mp_hands.HAND_CONNECTIONS,
#                 mp_drawing_styles.get_default_hand_landmarks_style(),
#                 mp_drawing_styles.get_default_hand_connections_style())
#         cv2.imwrite(
#             '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
#         # Draw hand world landmarks.
#         if not results.multi_hand_world_landmarks:
#             continue
#         for hand_world_landmarks in results.multi_hand_world_landmarks:
#             mp_drawing.plot_landmarks(
#                 hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)


# For webcam input:
rs = []
rs2 = []
filename = './train/video856387.avi'
cap = cv2.VideoCapture(filename)
# cap = cv2.VideoCapture(0)
with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
    with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
        while True:
            success, image = cap.read()
            if not success:
                # cap = cv2.VideoCapture('./train/video1058225.avi')
                # print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                break
                # cap = cv2.VideoCapture(filename)
                # continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            if results.pose_landmarks:
                rs.append([[lm.x, lm.y, lm.z]
                      for lm in results.pose_landmarks.landmark[11:23]])
            else:
                rs.append([[0,0,0] *12])
            # Draw the pose annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            results = hands.process(image)
            rs2.append(results.multi_hand_landmarks)
            # Draw the hand annotation on the image.
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(1) == ord('q'):
                break
cv2.destroyAllWindows()
cap.release()


In [83]:
IMAGE_FILES

[]

In [17]:
rs2

[None,
 None,
 None,
 None,
 None,
 [landmark {
    x: 0.3794522285461426
    y: 0.5555002689361572
    z: 3.625414990438003e-07
  }
  landmark {
    x: 0.36795905232429504
    y: 0.5033509731292725
    z: -0.0031796852126717567
  }
  landmark {
    x: 0.3769146203994751
    y: 0.441606342792511
    z: -0.01315216813236475
  }
  landmark {
    x: 0.40982070565223694
    y: 0.4002031087875366
    z: -0.023888999596238136
  }
  landmark {
    x: 0.4401116371154785
    y: 0.37519142031669617
    z: -0.03339827433228493
  }
  landmark {
    x: 0.34761953353881836
    y: 0.41626229882240295
    z: -0.021510319784283638
  }
  landmark {
    x: 0.3901189863681793
    y: 0.3514750599861145
    z: -0.03874848037958145
  }
  landmark {
    x: 0.4239751994609833
    y: 0.33343183994293213
    z: -0.048657793551683426
  }
  landmark {
    x: 0.4472663700580597
    y: 0.3307051360607147
    z: -0.052838828414678574
  }
  landmark {
    x: 0.3640817105770111
    y: 0.43654537200927734
    z: -0.0284

In [18]:
rs

[[[0.2303447425365448, 0.4169928729534149, -0.15855886042118073],
  [0.04930402338504791, 0.4701882004737854, -0.15099994838237762],
  [0.3144832253456116, 0.5642959475517273, -0.2722730338573456],
  [0.03564883768558502, 0.6632885336875916, -0.32856735587120056],
  [0.2764362394809723, 0.5216426849365234, -0.4833597242832184],
  [0.1067735105752945, 0.563593864440918, -0.6130067110061646],
  [0.26331666111946106, 0.5151395797729492, -0.5326244235038757],
  [0.1300528645515442, 0.5462297797203064, -0.6660051345825195],
  [0.25539064407348633, 0.4856139123439789, -0.5112150311470032],
  [0.12570394575595856, 0.5162898302078247, -0.6476827263832092],
  [0.2523299753665924, 0.49546608328819275, -0.47675853967666626],
  [0.12573352456092834, 0.5250808000564575, -0.6110976934432983]],
 [[0.23796887695789337, 0.42131051421165466, -0.2612573206424713],
  [0.04930287227034569, 0.47233283519744873, -0.25153058767318726],
  [0.31727102398872375, 0.5929280519485474, -0.30394572019577026],
  [0.03